In [ ]:
import networkx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._experimental import tree, graphviz

In [ ]:
module_dependencies = tree.get_module_dependencies()
internal_modules = set(module_dependencies.keys())

In [ ]:
root = 'pymedphys'

top_level_api = [item for item in module_dependencies[root] if not item[2].startswith('_')]
module_apis = [item[0] for item in top_level_api if item[0] == item[1]]

second_level_apis = {}
for module in module_apis:
    second_level_apis[module] = [item for item in module_dependencies[module] if not item[2].startswith('_')]
    
exposure_module_maps = {
    f"{root}.{item[2]}": item[1] for item in top_level_api if item[0] != item[1]
}

for module, second_level_api in second_level_apis.items():
    exposure_module_maps = {
        **exposure_module_maps,
        **{f"{module}.{item[2]}": item[1] for item in second_level_api}        
    }

exposure_module_maps

In [ ]:
def create_svg(api_name, module_name, href, module_dependencies, internal_modules):
    di_graph = networkx.DiGraph()
    di_graph.add_node(module_name)
    traversal_nodes = {module_name}
    
    while traversal_nodes:
        node = traversal_nodes.pop()
        raw_dependencies = module_dependencies[node]

        for dependency in raw_dependencies:
            if (
                not dependency[2].startswith('_') and
                not dependency[1] in di_graph and
                dependency[1] in internal_modules
            ):
                traversal_nodes.add(dependency[1])
                di_graph.add_node(dependency[1])
                di_graph.add_edge(node, dependency[1])
                
    edges = ""
    for edge in di_graph.edges:
        edges = edges + f'"{edge[0]}" -> "{edge[1]}";\n'
        
    graphviz.dot_string_to_svg(
        f"""
            digraph sample {{
                "{module_name}" [label="{api_name}"] [URL="{href}"];
                {edges}
            }}
        """, 
        f'{api_name}.svg'
    )

In [ ]:
api_name, module_name = list(exposure_module_maps.items())[0]
api_name, module_name

In [ ]:
exposure_module_maps

In [ ]:
href="https://github.com/pymedphys/pymedphys/tree/main/lib/pymedphys"

In [ ]:
create_svg(api_name, module_name, href, module_dependencies, internal_modules)

In [ ]:
for api_name, module_name in exposure_module_maps.items():
    print(api_name)
    create_svg(api_name, module_name, href, module_dependencies, internal_modules)

In [ ]:
# internal_modules